This notebook will use craft to detect if text should be rotated prior to using tesseract.

In [3]:
import cv2
import matplotlib.pyplot as plt
import math 
import numpy as np 
from pathlib import Path 
import sys
sys.path.append(str(Path.cwd().parent.parent))
from utils import create_fh_logger

In [4]:
src = Path.cwd().parent.parent.parent.parent / 'processing' / 'nro_declassified' / 'imgs'
dst = src.parent / 'cleaned_imgs'
dst.mkdir(parents=True, exist_ok=True)
east_weights = dst.parent.parent / 'models' / 'frozen_east_text_detection.pb'
logs = src.parent.parent / 'logs'
logs.mkdir(exist_ok=True)
logger = create_fh_logger(logs / "cleaned_imgs.log")

### Code to Identify Rotations: 0, 90, 180, 270
The code below uses EDGE text detection to identify text for an image un-rotated, and then rotated: 90, 180, or 270 degrees. Then, non-maximum supression to filter out duplicative detections. Lastly, it does a count on detections for that rotation. 

The detection count per rotation will be used to identify the ideal rotation for the entire pdf. Since mirrored rotations perform similarly, a rotation will only be performed if the rotated detections are at least 10% more.

In [5]:
# https://github.com/opencv/opencv/blob/7fb70e170154d064ef12d8fec61c0ae70812ce3d/samples/dnn/text_detection.py
def decode(scores, geometry, scoreThresh):
    detections = []
    confidences = []
    height = scores.shape[2]
    width = scores.shape[3]
    for y in range(0, height):
        scoresData = scores[0][0][y]
        x0_data = geometry[0][0][y]
        x1_data = geometry[0][1][y]
        x2_data = geometry[0][2][y]
        x3_data = geometry[0][3][y]
        anglesData = geometry[0][4][y]
        for x in range(0, width):
            score = scoresData[x]
            if(score < scoreThresh):
                continue
            offsetX = x * 4.0
            offsetY = y * 4.0
            angle = anglesData[x]
            cosA = math.cos(angle)
            sinA = math.sin(angle)
            h = x0_data[x] + x2_data[x]
            w = x1_data[x] + x3_data[x]
            offset = ([offsetX + cosA * x1_data[x] + sinA * x2_data[x], offsetY - sinA * x1_data[x] + cosA * x2_data[x]])
            p1 = (-sinA * h + offset[0], -cosA * h + offset[1])
            p3 = (-cosA * w + offset[0],  sinA * w + offset[1])
            center = (0.5*(p1[0]+p3[0]), 0.5*(p1[1]+p3[1]))
            detections.append((center, (w,h), -1*angle * 180.0 / math.pi))
            confidences.append(float(score))
    return [detections, confidences]

In [9]:
resize_hw = 640
conf_threshold = .2
nms_threshold=.2
available_rotations = [cv2.ROTATE_180, cv2.ROTATE_90_CLOCKWISE, cv2.ROTATE_90_COUNTERCLOCKWISE, -1]
rotations = {key: 0 for key in available_rotations} # lets use this as an accumulator to count the amount of text matches per rotation to be used as our final determinator for how to rotate the image
net = cv2.dnn.readNetFromTensorflow(str(east_weights))
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
outNames = ["feature_fusion/Conv_7/Sigmoid", "feature_fusion/concat_3"]
for image in list(Path(rotation_test).glob('*png')):
    name = image.name
    image = cv2.imread(str(image))
    for rotation in available_rotations:
        frame = image.copy()
        frame = cv2.rotate(frame, rotation) if rotation >=0 else frame
        height_ = frame.shape[0]
        width_ = frame.shape[1]
        rW = width_ / float(resize_hw)
        rH = height_ / float(resize_hw)
        blob = cv2.dnn.blobFromImage(frame, 1.0, (resize_hw, resize_hw), (123.68, 116.78, 103.94), True, False)
        net.setInput(blob)
        outs = net.forward(outNames)

        # Get scores and geometry
        scores = outs[0]
        geometry = outs[1]
        [boxes, confidences] = decode(scores, geometry, conf_threshold)
        indices = cv2.dnn.NMSBoxesRotated(boxes, confidences, conf_threshold, nms_threshold)
        rotations[rotation] += len(indices)